## Importaciones

In [ ]:
import pandas as pd
import os
import numpy as np

## Cargar Datasets

In [21]:

# 📌 Ruta al directorio de datasets
data_path = r"C:\Users\dalarroy\surf-prediction-project\data\bronze"

# 📌 Obtener todos los archivos .xlsx en la carpeta
files = [f for f in os.listdir(data_path) if f.endswith(".xlsx")]

# 📌 Cargar cada archivo como un DataFrame con el formato `df_{nombre}`
for file in files:
    file_name = file.split('.')[0]  # Obtener el nombre del archivo sin extensión
    df_variable_name = f"df_{file_name}"  # Crear el nombre de la variable con el prefijo "df_"
    
    globals()[df_variable_name] = pd.read_excel(os.path.join(data_path, file))  # Crear el DataFrame con el nombre ajustado
    
    # 📌 Mostrar información básica del DataFrame
    print(f"\n📌 Dataset cargado: {df_variable_name} ({file})")





📌 Dataset cargado: df_bio_pysurfline (bio_pysurfline.xlsx)

📌 Dataset cargado: df_bio_scrap_surfline (bio_scrap_surfline.xlsx)

📌 Dataset cargado: df_pg_pysurfline (pg_pysurfline.xlsx)

📌 Dataset cargado: df_pg_scrap_surfline (pg_scrap_surfline.xlsx)

📌 Dataset cargado: df_yatch_pysurfline (yatch_pysurfline.xlsx)

📌 Dataset cargado: df_yatch_scrap_surfline (yatch_scrap_surfline.xlsx)


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\dalarroy\\surf-prediction-project\\data\\bronze\\~$bio_pysurfline.xlsx'

## Columnas

In [22]:
df_bio_pysurfline .columns

Index(['timestamp_dt', 'timestamp_timestamp', 'probability', 'utcOffset',
       'surf_min', 'surf_max', 'surf_optimalScore', 'surf_plus',
       'surf_humanRelation', 'surf_raw_min', 'surf_raw_max', 'power',
       'swells_0_height', 'swells_0_period', 'swells_0_impact',
       'swells_0_power', 'swells_0_direction', 'swells_0_directionMin',
       'swells_0_optimalScore', 'swells_1_height', 'swells_1_period',
       'swells_1_impact', 'swells_1_power', 'swells_1_direction',
       'swells_1_directionMin', 'swells_1_optimalScore', 'swells_2_height',
       'swells_2_period', 'swells_2_impact', 'swells_2_power',
       'swells_2_direction', 'swells_2_directionMin', 'swells_2_optimalScore',
       'swells_3_height', 'swells_3_period', 'swells_3_impact',
       'swells_3_power', 'swells_3_direction', 'swells_3_directionMin',
       'swells_3_optimalScore', 'swells_4_height', 'swells_4_period',
       'swells_4_impact', 'swells_4_power', 'swells_4_direction',
       'swells_4_directionMin

## Ranking de Swells 

Se calcula el puntaje de cada swell en función de:
- Altura
- Período
- Potencia 
- Impacto

Se asignan pesos específicos a cada factor. Se determinan los tres swells más relevantes en cada timestamp: `primary_swell`, `secondary_swell` y `third_swell` basándose en sus puntajes. 
Si solo hay uno o dos swells, los valores secundarios quedarán como `None`.

El cálculo se aplica a los 3 dataframes de pysurfline.


In [37]:
import pandas as pd

# 📌 Factores y pesos ajustados (sin optimalScore)
weights = {
    "height": 0.4,
    "period": 0.25,
    "power": 0.25,
    "impact": 0.1
}

def calculate_swell_scores(df):
    """ 
    Calcula un puntaje para cada swell en cada timestamp sin optimalScore, ignorando NaN.
    También asigna primary_swell, secondary_swell y third_swell.
    """
    df = df.copy()
    timestamps = df["timestamp_dt"]

    swell_scores = []
    
    for i in range(6):
        score = (
            df[f"swells_{i}_height"].fillna(0) * weights["height"] +
            df[f"swells_{i}_period"].fillna(0) * weights["period"] +
            df[f"swells_{i}_power"].fillna(0) * weights["power"] +
            df[f"swells_{i}_impact"].fillna(0) * weights["impact"]
        )
        swell_scores.append(score)

    df_swells_scores = pd.DataFrame(swell_scores).T
    df_swells_scores.columns = [f"swell_{i}_score" for i in range(6)]
    df_swells_scores.insert(0, "timestamp_dt", timestamps)

    # 📌 Determinar primary swell
    primary_swell_idx = df_swells_scores.iloc[:, 1:].idxmax(axis=1)
    primary_swell_score = df_swells_scores.iloc[:, 1:].max(axis=1)

    # 📌 Encontrar secondary y third swell asegurando que haya más de un swell presente
    def find_next_swell(row, exclude=[]):
        """ Encuentra el swell con mayor puntaje excluyendo los ya seleccionados """
        filtered_row = row.drop(exclude, errors="ignore")
        return filtered_row.idxmax() if not filtered_row.empty else None

    def find_next_swell_score(row, exclude=[]):
        """ Encuentra el puntaje del swell con mayor puntaje excluyendo los ya seleccionados """
        filtered_row = row.drop(exclude, errors="ignore")
        return filtered_row.max() if not filtered_row.empty else None

    secondary_swell_idx = df_swells_scores.iloc[:, 1:].apply(lambda row: find_next_swell(row, [primary_swell_idx[row.name]]), axis=1)
    secondary_swell_score = df_swells_scores.iloc[:, 1:].apply(lambda row: find_next_swell_score(row, [primary_swell_idx[row.name]]), axis=1)

    third_swell_idx = df_swells_scores.iloc[:, 1:].apply(lambda row: find_next_swell(row, [primary_swell_idx[row.name], secondary_swell_idx[row.name]]), axis=1)
    third_swell_score = df_swells_scores.iloc[:, 1:].apply(lambda row: find_next_swell_score(row, [primary_swell_idx[row.name], secondary_swell_idx[row.name]]), axis=1)

    df_swells_scores["primary_swell"] = primary_swell_idx
    df_swells_scores["primary_swell_score"] = primary_swell_score
    df_swells_scores["secondary_swell"] = secondary_swell_idx
    df_swells_scores["secondary_swell_score"] = secondary_swell_score
    df_swells_scores["third_swell"] = third_swell_idx
    df_swells_scores["third_swell_score"] = third_swell_score

    return df_swells_scores

# 📌 Aplicar la función a cada DataFrame
df_swell_score_bio_pysurfline = calculate_swell_scores(df_bio_pysurfline)
df_swell_score_pg_pysurfline = calculate_swell_scores(df_pg_pysurfline)
df_swell_score_yatch_pysurfline = calculate_swell_scores(df_yatch_pysurfline)


In [38]:
df_swell_score_bio_pysurfline.head()

,timestamp_dt,swell_0_score,swell_1_score,swell_2_score,swell_3_score,swell_4_score,swell_5_score,primary_swell,primary_swell_score,secondary_swell,secondary_swell_score,third_swell,third_swell_score
0,2024-12-02 06:00:00,2.778,5.379567,0.0,0.0,0.0,0.0,swell_1_score,5.379567,swell_0_score,2.778,swell_2_score,0.0
1,2024-12-02 12:00:00,3.144,4.398707,0.0,0.0,0.0,0.0,swell_1_score,4.398707,swell_0_score,3.144,swell_2_score,0.0
2,2024-12-02 18:00:00,3.410,5.167262,0.0,0.0,0.0,0.0,swell_1_score,5.167262,swell_0_score,3.410,swell_2_score,0.0
3,2024-12-03 06:00:00,3.542,6.338490,0.0,0.0,0.0,0.0,swell_1_score,6.338490,swell_0_score,3.542,swell_2_score,0.0
4,2024-12-03 12:00:00,3.318,5.908952,0.0,0.0,0.0,0.0,swell_1_score,5.908952,swell_0_score,3.318,swell_2_score,0.0


In [39]:
df_swell_score_pg_pysurfline.head()

,timestamp_dt,swell_0_score,swell_1_score,swell_2_score,swell_3_score,swell_4_score,swell_5_score,primary_swell,primary_swell_score,secondary_swell,secondary_swell_score,third_swell,third_swell_score
0,2024-12-01 06:00:00,2.252005,13.421350,1.318,2.613388,0.0,3.181680,swell_1_score,13.421350,swell_5_score,3.181680,swell_3_score,2.613388
1,2024-12-01 12:00:00,1.790000,9.627483,1.274,2.412593,0.0,3.153868,swell_1_score,9.627483,swell_5_score,3.153868,swell_3_score,2.412593
2,2024-12-01 18:00:00,1.810000,6.710303,0.000,0.000000,0.0,2.940065,swell_1_score,6.710303,swell_5_score,2.940065,swell_0_score,1.810000
3,2024-12-02 06:00:00,0.000000,4.565263,0.000,0.000000,0.0,136.107155,swell_5_score,136.107155,swell_1_score,4.565263,swell_0_score,0.000000
4,2024-12-02 12:00:00,0.000000,3.309650,0.000,0.000000,0.0,381.124600,swell_5_score,381.124600,swell_1_score,3.309650,swell_0_score,0.000000
